In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import folium

warnings.filterwarnings(action='ignore')
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.options.display.float_format="{:.5f}".format
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
data = pd.read_csv('data/data.csv')
#submission = pd.read_csv('data/submission.csv')
target = pd.read_csv('data/target.csv')
#student = pd.read_csv('data/인천_학령인구_추계.csv', encoding = 'cp949')

In [3]:
data.head()

,수요지 id,X,Y,초등학령인구
0,C1,126.37150,37.46378,1
1,C2,126.47950,37.49578,46
2,C3,126.48150,37.49978,36
3,C4,126.48250,37.49578,81
4,C5,126.48350,37.49778,25


In [4]:
data.columns = ['수요지 id', '수요_X', '수요_Y', '초등학령인구']

In [5]:
data.head()

,수요지 id,수요_X,수요_Y,초등학령인구
0,C1,126.37150,37.46378,1
1,C2,126.47950,37.49578,46
2,C3,126.48150,37.49978,36
3,C4,126.48250,37.49578,81
4,C5,126.48350,37.49778,25


In [6]:
target.head()

,학교 id,X,Y
0,F1,126.54392,37.46729
1,F2,126.56873,37.49630
2,F3,126.49277,37.48443
3,F4,126.62968,37.49578
4,F5,126.52829,37.45028


In [7]:
target.columns = ['학교 id', '공급_X', '공급_Y']

In [8]:
target.head()

,학교 id,공급_X,공급_Y
0,F1,126.54392,37.46729
1,F2,126.56873,37.49630
2,F3,126.49277,37.48443
3,F4,126.62968,37.49578
4,F5,126.52829,37.45028


In [9]:
sum_df = pd.DataFrame()

# 반복문을 통해 df1 각 행에 df2를 붙이기
for _, row in target.iterrows():
    temp_df = pd.concat([pd.DataFrame(row).T] * len(data), ignore_index=True)
    temp_df = temp_df.join(data)
    sum_df = pd.concat([sum_df, temp_df])

sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,초등학령인구
0,F1,126.54392,37.46729,C1,126.37150,37.46378,1
1,F1,126.54392,37.46729,C2,126.47950,37.49578,46
2,F1,126.54392,37.46729,C3,126.48150,37.49978,36
3,F1,126.54392,37.46729,C4,126.48250,37.49578,81
4,F1,126.54392,37.46729,C5,126.48350,37.49778,25
...,...,...,...,...,...,...,...
306,F50,126.50114,37.51461,C307,126.65150,37.48278,2
307,F50,126.50114,37.51461,C308,126.65250,37.47278,12
308,F50,126.50114,37.51461,C309,126.65250,37.47978,1
309,F50,126.50114,37.51461,C310,126.65250,37.48078,1


In [10]:
sum_df = sum_df.drop('초등학령인구', axis = 1)

In [11]:
sum_df['위도_범위'] = sum_df.apply(lambda row: sorted([row['수요_Y'], row['공급_Y']]), axis=1)

In [12]:
sum_df['경도_범위'] = sum_df.apply(lambda row: sorted([row['수요_X'], row['공급_X']]), axis=1)

In [13]:
sum_df.head()

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]"
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]"
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]"
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]"
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]"


In [14]:
sum_df['CCTV_개수'] = 0

In [15]:
sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위,CCTV_개수
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]",0
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]",0
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]",0
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]",0
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]",0
...,...,...,...,...,...,...,...,...,...
306,F50,126.50114,37.51461,C307,126.65150,37.48278,"[37.4827824, 37.514612976919096]","[126.5011352571878, 126.6515048]",0
307,F50,126.50114,37.51461,C308,126.65250,37.47278,"[37.4727824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
308,F50,126.50114,37.51461,C309,126.65250,37.47978,"[37.4797824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
309,F50,126.50114,37.51461,C310,126.65250,37.48078,"[37.4807824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0


In [16]:
sum_df.reset_index(drop=True, inplace=True)

In [17]:
sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위,CCTV_개수
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]",0
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]",0
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]",0
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]",0
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]",0
...,...,...,...,...,...,...,...,...,...
15545,F50,126.50114,37.51461,C307,126.65150,37.48278,"[37.4827824, 37.514612976919096]","[126.5011352571878, 126.6515048]",0
15546,F50,126.50114,37.51461,C308,126.65250,37.47278,"[37.4727824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
15547,F50,126.50114,37.51461,C309,126.65250,37.47978,"[37.4797824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
15548,F50,126.50114,37.51461,C310,126.65250,37.48078,"[37.4807824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0


In [18]:
warning = pd.read_csv('data/CCTV_위경도.csv', encoding = 'cp949')

In [19]:
warning.head()

,번호,관리기관명,소재지도로명주소,소재지지번주소,설치목적구분,카메라대수,카메라화소수,촬영방면정보,보관일수,설치연월,관리기관전화번호,WGS84위도,WGS84경도,데이터기준일자
0,1,인천광역시 동구청,인천광역시 동구 배송로 2,인천광역시 동구 송림동 209-16,다목적,3,200.00000,360도전방면,30,2014-09,032-770-6300,37.47413,126.63775,2020-03-02
1,2,인천광역시 동구보건소,인천광역시 동구 석수로 90,인천광역시 동구 만석동 18-3,시설물관리,8,200.00000,360도 전방면,30,2016-04,032-770-5705,37.48300,126.62590,2020-06-18
2,4,인천광역시 동구치매안심센터,"인천광역시 동구 송림로 113, 송영빌딩 2층 (송림동)",인천광역시 동구 송림동 49번지 13호 송영빌딩,생활방범,5,240.00000,90도단방향,30,2018-11,032-760-4929,37.47691,126.64721,2020-06-23
3,6,인천광역시 동구 송현1·2동 행정복지센터,인천광역시 동구 수문통로 26,인천광역시 동구 송현동 67-84,기타,4,NaN,NaN,15,2020-02,032-770-5820,37.47849,126.63338,2020-06-23
4,9,인천광역시 동구청,인천광역시 동구 화수로7,인천광역시 동구 송현동 60,어린이보호,1,200.00000,360도전방면,30,2015-01,032-770-6300,37.48030,126.63600,2020-03-02


In [20]:
from tqdm import tqdm
import time

In [21]:
for i in tqdm(range(len(sum_df))):
    time.sleep(0.01)
    for j in range(len(warning)):
        if (sum_df.iloc[i, 6][0] <= warning.iloc[j,11] <= sum_df.iloc[i, 6][1]) and (sum_df.iloc[i, 7][0] <= warning.iloc[j,12] <= sum_df.iloc[i, 7][1]):
            sum_df.at[i, 'CCTV_개수'] += 1

100%|████████████████████████████████████████████████████████████████████████████| 15550/15550 [22:24<00:00, 11.56it/s]


In [22]:
sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위,CCTV_개수
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]",1
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]",83
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]",102
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]",82
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]",92
...,...,...,...,...,...,...,...,...,...
15545,F50,126.50114,37.51461,C307,126.65150,37.48278,"[37.4827824, 37.514612976919096]","[126.5011352571878, 126.6515048]",236
15546,F50,126.50114,37.51461,C308,126.65250,37.47278,"[37.4727824, 37.514612976919096]","[126.5011352571878, 126.6525048]",804
15547,F50,126.50114,37.51461,C309,126.65250,37.47978,"[37.4797824, 37.514612976919096]","[126.5011352571878, 126.6525048]",385
15548,F50,126.50114,37.51461,C310,126.65250,37.48078,"[37.4807824, 37.514612976919096]","[126.5011352571878, 126.6525048]",324


In [23]:
sum_df['CCTV_개수'].value_counts()

0       328
1       225
2       215
4       168
6       149
       ... 
1128      1
1140      1
1157      1
1163      1
832       1
Name: CCTV_개수, Length: 1033, dtype: int64

In [24]:
sum_df.to_csv('CCTV_weight.csv', index=False)